In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight

In [2]:
df = pd.read_csv('labeled_data.csv', index_col=0)
df.head(5)

,sentence,prediction
0,twitter prior musk takeover talking directly n...,1
1,article say imply states feature turns crashes...,1
2,og musk duck lives wall,1
3,dare_speak way great powerful musk obviously d...,1
4,cannot_wait finally excuse shower douche,1


In [3]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [4]:
for label, count in dict(df['prediction'].value_counts()).items():
    print(f'{label}: {count / df.shape[0]}')

1: 0.9035761589403973
0: 0.09642384105960265


In [5]:
df['prediction'] = df['prediction'].astype('int8')

In [6]:
df[df['prediction'] == 0].shape

(5824, 2)

In [7]:
# limit the dataset to fit in memory
limited_df = pd.concat([df[df['prediction'] == 0], df[df['prediction'] == 1][:10000]])
limited_df.shape

(15824, 2)

In [8]:
limited_df.sample(5)

,sentence,prediction
102,vaguely elon_musk aligned clickbait rich - guy,1
4870,context calling musk bully,1
3320,dm_matter old may open good ideas good referen...,0
700,tl_; dr : maybe really understand point podcas...,1
5335,musk become fat oligarch,1


In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    limited_df['sentence'], limited_df['prediction'], test_size=0.1, stratify=limited_df['prediction'])

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.33, stratify=y_train
)

In [10]:
for label, count in dict(y_test.value_counts()).items():
    print(f'{label}: {count / y_test.shape[0]}')

1: 0.6317119393556538
0: 0.36828806064434616


In [11]:
for label, count in dict(y_val.value_counts()).items():
    print(f'{label}: {count / y_val.shape[0]}')

1: 0.6319148936170212
0: 0.3680851063829787


In [12]:
for label, count in dict(y_test.value_counts()).items():
    print(f'{label}: {count / y_test.shape[0]}')

1: 0.6317119393556538
0: 0.36828806064434616


In [13]:
y_train.value_counts(), y_val.value_counts(), y_test.value_counts()

(1    6030
 0    3511
 Name: prediction, dtype: int64,
 1    2970
 0    1730
 Name: prediction, dtype: int64,
 1    1000
 0     583
 Name: prediction, dtype: int64)

In [14]:
MAX_FEATURES = 5000

count_vec = CountVectorizer(max_features=MAX_FEATURES, min_df=5)
x_train = count_vec.fit_transform(x_train).toarray()
x_val = count_vec.transform(x_val).toarray()
x_test = count_vec.transform(x_test).toarray()

In [15]:
import tensorflow as tf

with tf.device('/cpu:0'):
    x_train = tf.convert_to_tensor(x_train, np.int8)
    x_val = tf.convert_to_tensor(x_val, np.int8)
    x_test = tf.convert_to_tensor(x_test, np.int8)

In [16]:
x_train.shape

TensorShape([9541, 5000])

In [17]:
from keras.layers import Dense, Dropout, LSTM
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
from keras.metrics import Recall

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

In [18]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

{0: 1.358729706636286, 1: 0.7911276948590381}

In [19]:
from keras.backend import clear_session

EPOCHS = 25
BATCH_SIZE = 8

def compile_and_fit_model(model: Model, model_name: str):
    clear_session()

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', Recall()]  # class imbalance
    )

    model_history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping],
        class_weight=class_weights
    )

    model.save(model_name + '.h5')

    return model_history.history

In [20]:
def display_confusion_matrix(model_prediction):
    model_prediction = np.argmax(model_prediction, axis=1)
    print(confusion_matrix(y_test, model_prediction, normalize='pred'))

In [21]:
def display_classification_report(model_prediction):
    model_prediction = np.argmax(model_prediction, axis=1)
    print(classification_report(y_test, model_prediction, zero_division=0))

In [22]:
def display_model_results(model: Model):
    prediction = model.predict(x_test)

    print('Confussion matrix:')
    display_confusion_matrix(prediction)
    
    print('\nClassification report:')
    display_classification_report(prediction)

    print('\nModel evaluation on test data [loss, accuracy, recall]:')
    print(model.evaluate(x_test, y_test))

In [23]:
dense_model1 = Sequential([
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid'),
])

dense_model1_history = compile_and_fit_model(dense_model1, 'dense_model1')

Epoch 1/25
1193/1193 [==============================] - 7s 4ms/step - loss: 0.2949 - accuracy: 0.8920 - recall: 0.9201 - val_loss: 0.2361 - val_accuracy: 0.9081 - val_recall: 0.9084
Epoch 2/25
1193/1193 [==============================] - 4s 4ms/step - loss: 0.1260 - accuracy: 0.9473 - recall: 0.9489 - val_loss: 0.2984 - val_accuracy: 0.9157 - val_recall: 0.9279
Epoch 3/25
1193/1193 [==============================] - 4s 4ms/step - loss: 0.0622 - accuracy: 0.9705 - recall: 0.9653 - val_loss: 0.5366 - val_accuracy: 0.9077 - val_recall: 0.9044
Epoch 4/25
1193/1193 [==============================] - 4s 4ms/step - loss: 0.0522 - accuracy: 0.9755 - recall: 0.9680 - val_loss: 0.6225 - val_accuracy: 0.9119 - val_recall: 0.9286
Epoch 5/25
1193/1193 [==============================] - 4s 4ms/step - loss: 0.0355 - accuracy: 0.9801 - recall: 0.9715 - val_loss: 0.6380 - val_accuracy: 0.9115 - val_recall: 0.9293
Epoch 6/25
1193/1193 [==============================] - 4s 4ms/step - loss: 0.0303 - accur

In [24]:
display_model_results(dense_model1)

50/50 [==============================] - 0s 861us/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 0s 2ms/step - loss: 0.2279 - accuracy: 0.9109 - recall: 0.9150
[0.22786106169223785, 0.9109286069869995, 0.9150000214576721]


In [25]:
dense_model2 = Sequential([
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid'),
])

dense_model2_history = compile_and_fit_model(dense_model2, 'dense_model2')

Epoch 1/25
1193/1193 [==============================] - 6s 4ms/step - loss: 0.3038 - accuracy: 0.8828 - recall: 0.8964 - val_loss: 0.2419 - val_accuracy: 0.9149 - val_recall: 0.9316
Epoch 2/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.1257 - accuracy: 0.9503 - recall: 0.9532 - val_loss: 0.3182 - val_accuracy: 0.9187 - val_recall: 0.9455
Epoch 3/25
1193/1193 [==============================] - 4s 4ms/step - loss: 0.0649 - accuracy: 0.9708 - recall: 0.9680 - val_loss: 0.5739 - val_accuracy: 0.9126 - val_recall: 0.9471
Epoch 4/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0483 - accuracy: 0.9773 - recall: 0.9698 - val_loss: 0.5764 - val_accuracy: 0.9123 - val_recall: 0.9620
Epoch 5/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0446 - accuracy: 0.9785 - recall: 0.9720 - val_loss: 0.8806 - val_accuracy: 0.9055 - val_recall: 0.9232
Epoch 6/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0309 - accur

In [26]:
display_model_results(dense_model2)

50/50 [==============================] - 0s 985us/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 0s 2ms/step - loss: 0.2184 - accuracy: 0.9172 - recall: 0.9380
[0.2183535099029541, 0.9172457456588745, 0.9380000233650208]


In [27]:
dense_model3 = Sequential([
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

dense_model3_history = compile_and_fit_model(dense_model3, 'dense_model3')

Epoch 1/25
1193/1193 [==============================] - 6s 4ms/step - loss: 0.3013 - accuracy: 0.8903 - recall: 0.9040 - val_loss: 0.2636 - val_accuracy: 0.9126 - val_recall: 0.9118
Epoch 2/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.1236 - accuracy: 0.9501 - recall: 0.9556 - val_loss: 0.2875 - val_accuracy: 0.9151 - val_recall: 0.9202
Epoch 3/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0682 - accuracy: 0.9705 - recall: 0.9682 - val_loss: 0.5676 - val_accuracy: 0.9070 - val_recall: 0.9088
Epoch 4/25
1193/1193 [==============================] - 5s 4ms/step - loss: 0.0443 - accuracy: 0.9762 - recall: 0.9682 - val_loss: 0.7531 - val_accuracy: 0.9130 - val_recall: 0.9212
Epoch 5/25
1193/1193 [==============================] - 4s 3ms/step - loss: 0.0341 - accuracy: 0.9800 - recall: 0.9728 - val_loss: 1.1528 - val_accuracy: 0.8977 - val_recall: 0.8822
Epoch 6/25
1193/1193 [==============================] - 4s 3ms/step - loss: 0.0334 - accur

In [28]:
display_model_results(dense_model3)

50/50 [==============================] - 0s 831us/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 0s 1ms/step - loss: 0.2491 - accuracy: 0.9097 - recall: 0.9100
[0.24906308948993683, 0.9096651673316956, 0.9100000262260437]


In [40]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(num_words=MAX_FEATURES, split=' ')
tokenizer.fit_on_texts(limited_df['sentence'].values)
X = tokenizer.texts_to_sequences(limited_df['sentence'].values)
X = pad_sequences(X, maxlen=MAX_FEATURES, padding='post')

In [41]:
x_train, x_test, y_train, y_test = train_test_split(
    X, limited_df['prediction'], test_size=0.1, stratify=limited_df['prediction'])

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.33, stratify=y_train)

In [42]:
with tf.device('/cpu:0'):
    x_train = tf.convert_to_tensor(x_train, np.int32)
    x_val = tf.convert_to_tensor(x_val, np.int32)
    x_test = tf.convert_to_tensor(x_test, np.int32)

In [43]:
embedding_dimension = 64
lstm_output = 64

In [45]:
from keras.layers import Embedding

lstm_model1 = Sequential([
    Embedding(MAX_FEATURES, embedding_dimension, input_length=X.shape[1]),
    LSTM(lstm_output, dropout=0.2),
    Dense(1, activation='sigmoid')
])

lstm_model1_history = compile_and_fit_model(lstm_model1, 'lstm_model1')

Epoch 1/25
1193/1193 [==============================] - 181s 150ms/step - loss: 0.6937 - accuracy: 0.4786 - recall: 0.4355 - val_loss: 0.6934 - val_accuracy: 0.3681 - val_recall: 0.0000e+00
Epoch 2/25
1193/1193 [==============================] - 157s 132ms/step - loss: 0.6932 - accuracy: 0.4515 - recall: 0.3083 - val_loss: 0.6810 - val_accuracy: 0.6319 - val_recall: 1.0000
Epoch 3/25
1193/1193 [==============================] - 161s 135ms/step - loss: 0.6935 - accuracy: 0.5607 - recall: 0.7254 - val_loss: 0.6929 - val_accuracy: 0.6319 - val_recall: 1.0000
Epoch 4/25
1193/1193 [==============================] - 166s 140ms/step - loss: 0.6933 - accuracy: 0.5125 - recall: 0.5584 - val_loss: 0.6924 - val_accuracy: 0.6319 - val_recall: 1.0000
Epoch 5/25
1193/1193 [==============================] - 167s 140ms/step - loss: 0.6933 - accuracy: 0.5264 - recall: 0.5887 - val_loss: 0.6907 - val_accuracy: 0.6319 - val_recall: 1.0000
Epoch 6/25
1193/1193 [==============================] - 161s 135ms

In [46]:
display_model_results(lstm_model1)

50/50 [==============================] - 3s 48ms/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 3s 58ms/step - loss: 0.6810 - accuracy: 0.6317 - recall: 1.0000
[0.6810417771339417, 0.6317119598388672, 1.0]


In [47]:
from keras.layers import Bidirectional

lstm_model2 = Sequential([
    Embedding(MAX_FEATURES, embedding_dimension, input_length=X.shape[1]),
    Bidirectional(LSTM(lstm_output, dropout=0.2)),
    Dense(1, activation='sigmoid')
])

lstm_model2_history = compile_and_fit_model(lstm_model2, 'lstm_model2')

Epoch 1/25
1193/1193 [==============================] - 329s 275ms/step - loss: 0.2716 - accuracy: 0.8944 - recall: 0.9114 - val_loss: 0.1947 - val_accuracy: 0.9281 - val_recall: 0.9418
Epoch 2/25
1193/1193 [==============================] - 296s 248ms/step - loss: 0.1432 - accuracy: 0.9499 - recall: 0.9575 - val_loss: 0.1952 - val_accuracy: 0.9264 - val_recall: 0.9505
Epoch 3/25
1193/1193 [==============================] - 285s 239ms/step - loss: 0.1023 - accuracy: 0.9637 - recall: 0.9660 - val_loss: 0.2024 - val_accuracy: 0.9262 - val_recall: 0.9498
Epoch 4/25
1193/1193 [==============================] - 292s 245ms/step - loss: 0.0738 - accuracy: 0.9746 - recall: 0.9751 - val_loss: 0.2359 - val_accuracy: 0.9194 - val_recall: 0.9226
Epoch 5/25
1193/1193 [==============================] - 289s 242ms/step - loss: 0.0556 - accuracy: 0.9791 - recall: 0.9793 - val_loss: 0.2829 - val_accuracy: 0.9249 - val_recall: 0.9320
Epoch 6/25
1193/1193 [==============================] - 299s 251ms/ste

In [48]:
display_model_results(lstm_model2)

50/50 [==============================] - 5s 98ms/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 5s 106ms/step - loss: 0.1796 - accuracy: 0.9223 - recall: 0.9380
[0.1795957386493683, 0.9222994446754456, 0.9380000233650208]


In [49]:
embedding_dimension = 96
lstm_output = 128

In [50]:
lstm_model3 = Sequential([
    Embedding(MAX_FEATURES, embedding_dimension, input_length=X.shape[1]),
    Bidirectional(LSTM(lstm_output, dropout=0.2)),
    Dense(1, activation='sigmoid')
])

lstm_model3_history = compile_and_fit_model(lstm_model3, 'lstm_model3')

Epoch 1/25
1193/1193 [==============================] - 356s 297ms/step - loss: 0.2897 - accuracy: 0.8908 - recall: 0.9090 - val_loss: 0.1920 - val_accuracy: 0.9262 - val_recall: 0.9384
Epoch 2/25
1193/1193 [==============================] - 332s 278ms/step - loss: 0.1465 - accuracy: 0.9505 - recall: 0.9580 - val_loss: 0.2137 - val_accuracy: 0.9281 - val_recall: 0.9357
Epoch 3/25
1193/1193 [==============================] - 332s 279ms/step - loss: 0.1061 - accuracy: 0.9644 - recall: 0.9692 - val_loss: 0.2013 - val_accuracy: 0.9251 - val_recall: 0.9444
Epoch 4/25
1193/1193 [==============================] - 330s 276ms/step - loss: 0.0799 - accuracy: 0.9707 - recall: 0.9731 - val_loss: 0.2379 - val_accuracy: 0.9236 - val_recall: 0.9340
Epoch 5/25
1193/1193 [==============================] - 339s 284ms/step - loss: 0.0678 - accuracy: 0.9751 - recall: 0.9758 - val_loss: 0.2654 - val_accuracy: 0.9143 - val_recall: 0.9189
Epoch 6/25
1193/1193 [==============================] - 333s 279ms/ste

In [51]:
display_model_results(lstm_model3)

50/50 [==============================] - 6s 104ms/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 6s 119ms/step - loss: 0.1748 - accuracy: 0.9267 - recall: 0.9440
[0.17477327585220337, 0.9267213940620422, 0.9440000057220459]


In [52]:
lstm_model4 = Sequential([
    Embedding(MAX_FEATURES, embedding_dimension, input_length=X.shape[1]),
    Bidirectional(LSTM(lstm_output, dropout=0.3)),
    Dense(1, activation='sigmoid')
])

lstm_model4_history = compile_and_fit_model(lstm_model4, 'lstm_model4')

Epoch 1/25
1193/1193 [==============================] - 352s 293ms/step - loss: 0.3590 - accuracy: 0.8392 - recall: 0.8604 - val_loss: 0.1871 - val_accuracy: 0.9315 - val_recall: 0.9721
Epoch 2/25
1193/1193 [==============================] - 329s 276ms/step - loss: 0.1579 - accuracy: 0.9458 - recall: 0.9516 - val_loss: 0.1852 - val_accuracy: 0.9270 - val_recall: 0.9623
Epoch 3/25
1193/1193 [==============================] - 331s 278ms/step - loss: 0.2751 - accuracy: 0.8864 - recall: 0.8766 - val_loss: 0.2343 - val_accuracy: 0.9072 - val_recall: 0.9141
Epoch 4/25
1193/1193 [==============================] - 326s 274ms/step - loss: 0.1518 - accuracy: 0.9442 - recall: 0.9439 - val_loss: 0.2271 - val_accuracy: 0.9115 - val_recall: 0.9131
Epoch 5/25
1193/1193 [==============================] - 325s 272ms/step - loss: 0.1094 - accuracy: 0.9585 - recall: 0.9575 - val_loss: 0.2146 - val_accuracy: 0.9228 - val_recall: 0.9532
Epoch 6/25
1193/1193 [==============================] - 338s 283ms/ste

In [53]:
display_model_results(lstm_model4)

50/50 [==============================] - 5s 103ms/step
Confussion matrix:
[[0.36828806 0.        ]
 [0.63171194 0.        ]]

Classification report:
              precision    recall  f1-score   support

           0       0.37      1.00      0.54       583
           1       0.00      0.00      0.00      1000

    accuracy                           0.37      1583
   macro avg       0.18      0.50      0.27      1583
weighted avg       0.14      0.37      0.20      1583


Model evaluation on test data [loss, accuracy, recall]:
50/50 [==============================] - 6s 113ms/step - loss: 0.1691 - accuracy: 0.9274 - recall: 0.9620
[0.1690511852502823, 0.9273531436920166, 0.9620000123977661]
